# Setup -- Import

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


pl.Config.set_tbl_hide_dataframe_shape(True)
pl.Config.set_fmt_str_lengths(60)
pl.Config.set_tbl_rows(25)

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# dataset_drive_path = '/content/drive/MyDrive/cursus/uc'
dataset_drive_path = '..'

In [ ]:
model_tracking = pd.DataFrame(columns=[
    'Model',
    'Hyperparameters',
    'Preprocessing',
    'Training Accuracy',
    'Validation Accuracy',
    'Test Accuracy',
    'Comments'
])

model_tracking

In [ ]:
def add_entry(new_entry: dict) -> None:
    return pd.concat([model_tracking, new_entry], ignore_index=True)

# Data Exploration

In [ ]:
intents_test_path = f"{dataset_drive_path}/data/intents_test.csv"
intents_train_path = f"{dataset_drive_path}/data/intents_train.csv"
intents_train = pl.read_csv(intents_train_path, new_columns=['intent', 'label'])
intents_test = pl.read_csv(intents_test_path, new_columns=['intent'])

In [ ]:
intents_train

In [ ]:
intents_test

To understand the types of intents present in the training data, we can examine the unique values in the intent column.

In [ ]:
import seaborn as sns

def plot_intent_distribution(data):
    plt.figure(figsize=(10, 6))
    sns.countplot(data=data.to_pandas(), x='label', palette='Set2')
    plt.xticks(rotation=45)
    plt.xlabel('Intent Label')
    plt.ylabel('Count')
    plt.title('Distribution of Intent Labels')
    plt.show()

In [ ]:
plot_intent_distribution(intents_train)

In [ ]:
import re
from collections import Counter, OrderedDict

def tokenize(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

In [ ]:
from sklearn.model_selection import train_test_split

intents_train, intents_valid = train_test_split(intents_train, test_size=0.2)

In [ ]:
intents_train

In [ ]:
intents_valid

In [ ]:
plot_intent_distribution(intents_train)

In [ ]:
plot_intent_distribution(intents_valid)

In [ ]:
class Vocabulary:
    def __init__(self, token_counts, pad_token="<pad>", unk_token="<unk>"):
        self.token_counts = token_counts
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.stoi = {}  # string to index
        self.itos = []  # index to string

        self.build_vocab()

    def build_vocab(self):
        self.stoi[self.pad_token] = 0
        self.stoi[self.unk_token] = 1
        self.itos.append(self.pad_token)
        self.itos.append(self.unk_token)

        for token, count in self.token_counts.items():
            if token not in self.stoi:
                idx = len(self.itos)
                self.stoi[token] = idx
                self.itos.append(token)

    def __getitem__(self, token):
        return self.stoi.get(token, 1)  # return unk_token index if token not found

    def __len__(self):
        return len(self.itos)

In [ ]:
token_counts = Counter()
for sample in intents_train.iter_rows():
    tokens = tokenize(sample[0])
    token_counts.update(tokens)


sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ord_dict = OrderedDict(sorted_by_freq_tuples)
print('Vocab-size:', len(token_counts))

vocab = Vocabulary(token_counts)

print([vocab[token] for token in ['this', 'is', 'an', 'example']])

In [ ]:
print([vocab[token] for token in ['can', 'i', 'return', 'the', 'bike', 'i', 'purchased']])

In [ ]:
s = set(list(intents_train['label']))
len_labels = len(s)
labels2idx = {i: j for j, i in enumerate(s)}
idx2labels = {j: i for j, i in enumerate(s)}

text_pipeline = lambda x: [vocab[token] for token in tokenize(x)]
label_pipeline = lambda x: labels2idx[x]

labels2idx

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset


def collate_batch(batch, test_set=False):
    label_list, text_list, lengths = [], [], []
    for sample in batch:
        if test_set == False:
            label_list.append(sample[1])
        processed_text = torch.tensor(text_pipeline(sample[0]), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor([label_pipeline(l) for l in label_list], dtype=torch.int64)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

batch_size = 24
train_dl = DataLoader([row for row in intents_train.iter_rows()], batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader([row for row in intents_valid.iter_rows()], batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader([row for row in intents_test.iter_rows()], batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# Modeling

## RNN

In [ ]:
from dataclasses import dataclass

@dataclass
class RNNConfig:

    vocab_size: int = len(vocab)
    embed_dim: int = 32
    rnn_hidden_dim: int = 32
    fc_hidden_dim: int = 64
    output_dim: int = len_labels
    dropout: float = 0.5


config = RNNConfig()

In [ ]:
class RNN(nn.Module):

    def __init__(self, config: RNNConfig):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(config.vocab_size, config.embed_dim, padding_idx=0)
        self.rnn = nn.RNN(config.embed_dim, config.rnn_hidden_dim, batch_first=True)

        self.fc1 = nn.Linear(config.rnn_hidden_dim, config.fc_hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(config.fc_hidden_dim, config.output_dim)

    def forward(self, x, lengths):
        x = self.embedding(x)
        x = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        _, hidden = self.rnn(x)

        x = hidden[-1, :, :]  # or consider using the entire RNN output sequence
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch.utils
import torch.utils.data


def train(
    dl_loader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    model: torch.nn.Module,
    ):

    """ Train the model on the training dataset """

    total_loss, total_acc = 0, 0

    model.train()
    for text, label, lengths in dl_loader:
        optimizer.zero_grad()
        preds = model(text, lengths)
        loss = criterion(preds, label)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(preds, 1)
        total_loss += loss.item() * label.size(0)
        total_acc += (preds  == label).float().sum().item()
    return total_loss / len(dl_loader.dataset), total_acc / len(dl_loader.dataset)

def evaluate(
    dl_loader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    model: torch.nn.Module,
    ):

    """ Evaluate the model on the validation or test set """

    total_loss, total_acc = 0, 0

    model.eval()
    with torch.inference_mode():
        for text, label, lengths in dl_loader:
            preds = model(text, lengths)
            loss = criterion(preds, label)
            _, preds = torch.max(preds, 1)
            total_acc += (preds == label).float().sum().item()
            total_loss += loss.item() * label.size(0)
        return total_loss / len(dl_loader.dataset), total_acc / len(dl_loader.dataset)

In [ ]:
model = RNN(config).to(device)
print(model)

In [ ]:
num_epochs = 30
lr = 0.001
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
torch.manual_seed(1)


for epoch in range(num_epochs):
    train_loss, train_acc = train(train_dl, optimizer, criteria, model)
    valid_loss, valid_acc = evaluate(valid_dl, criteria, model)
    print(f'Epoch {epoch}  train loss: {train_loss:.4f} train accuracy: {train_acc:.4f}',  f' valid loss: {valid_loss:.4f} valid accuracy: {valid_acc:.4f}')

In [ ]:
rnn_entry = pd.DataFrame([{
    'Model': 'RNN',
    'Hyperparameters': f"Learning rate: {lr}, Hidden units: {config.rnn_hidden_dim}",
    'Preprocessing': 'Lowercase, Remove punctuation',
    'Training Accuracy': f"{train_acc}",
    'Validation Accuracy': f"{valid_acc}",
    'Test Accuracy': 'N/A',
    'Comments': 'First attempt, overfitting observed'
}])

model_tracking = add_entry(rnn_entry)
model_tracking

## LSTM

In [ ]:
from dataclasses import dataclass

@dataclass
class LSTMConfig:

    vocab_size: int = len(vocab)
    embed_dim: int = 32
    lstm_hidden_dim: int = 100
    fc_hidden_dim: int = 32
    output_dim: int = len_labels
    dropout: float = 0.5


config = LSTMConfig()
config

In [ ]:
class LSTM(nn.Module):
    def __init__(self, config: RNNConfig):
        super().__init__()
        self.embedding = nn.Embedding(config.vocab_size, config.embed_dim, padding_idx=0)
        self.rnn = nn.LSTM(config.embed_dim, config.lstm_hidden_dim, batch_first=True, bidirectional=True)
        self.rnn_dropout = nn.Dropout(config.dropout)  # Add dropout to RNN output

        self.fc_layers = nn.Sequential(
            nn.Linear(config.lstm_hidden_dim*2, config.fc_hidden_dim),
            nn.Dropout(config.dropout),
            nn.ReLU(),
            nn.Linear(config.fc_hidden_dim, config.output_dim)
        )

    def forward(self, x, lengths):
        x = self.embedding(x)
        x = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)

        _, (hidden, cell) = self.rnn(x)
        x = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        x = self.rnn_dropout(x)
        x = self.fc_layers(x)
        return x

In [ ]:
model = LSTM(config).to(device)
print(model)

In [ ]:
lr = 0.005
num_epochs = 30
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
torch.manual_seed(1)

best_train_acc, best_valid_acc = 0, 0

for epoch in range(num_epochs):
    train_loss, train_acc = train(train_dl, optimizer, criteria, model)
    valid_loss, valid_acc = evaluate(valid_dl, criteria, model)
    
    if valid_acc > best_valid_acc:
        best_train_acc = train_acc
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'best_model.pth')
    
    print(f'Epoch {epoch}  train loss: {train_loss:.4f} train accuracy: {train_acc:.4f}',  f' valid loss: {valid_loss:.4f} valid accuracy: {valid_acc:.4f}')


In [ ]:
lstm_entry = pd.DataFrame([{
    'Model': 'LSTM',
    'Hyperparameters': f"Learning rate: {lr}, Hidden units: {config.lstm_hidden_dim}, Epochs: {num_epochs}",
    'Preprocessing': 'Lowercase, Remove punctuation',
    'Training Accuracy': f"{best_train_acc}",
    'Validation Accuracy': f"{best_valid_acc}",
    'Test Accuracy': 'N/A',
    'Comments': 'First attempt, overfitting observed'
}])

model_tracking = add_entry(lstm_entry)
model_tracking

## BERT

In [ ]:
%%capture
! pip install datasets evaluate

In [ ]:
import datasets

# Load the dataset
bikeshop_dataset = datasets.load_dataset("m0saan/bikeshop")

# Split the "train" split into training and validation sets
train_validation_ds = bikeshop_dataset["train"].train_test_split(train_size=0.8, seed=42)

# Rename the default "test" split to "validation"
validation_ds = train_validation_ds.pop("test")

# Create a DatasetDict to hold our splits
full_dataset = datasets.DatasetDict()

# Add the training, validation, and test sets to the DatasetDict
full_dataset["train"] = train_validation_ds["train"]
full_dataset["validation"] = validation_ds
full_dataset["test"] = bikeshop_dataset["test"]

# Print the full dataset dict
full_dataset

In [ ]:
s = set(full_dataset['train']['intent'])
len_labels = len(s)
labels2idx = {i: j for j, i in enumerate(s)}
idx2labels = {j: i for j, i in enumerate(s)}

text_pipeline = lambda x: [vocab[token] for token in tokenize(x)]
label_pipeline = lambda x: labels2idx[x]

labels2idx

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    tokenized_inputs = tokenizer(example["text"])
    labels = [label_pipeline(l) for l in example["intent"]]
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = full_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", "intent"])
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=37, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len_labels)
model

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
import evaluate

metric = evaluate.load("precision")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute(average='micro')

In [ ]:
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

predictions

In [ ]:
predictions = predictions.cpu().numpy().tolist()
predictions[:10]

In [ ]:
pd.DataFrame([idx2labels[idx] for idx in predictions], full_dataset['test']['text'])

In [1]:
[idx2labels[idx] for idx in predictions]

NameError: name 'predictions' is not defined

In [ ]:
import pickle

# Save the model
with open('/content/drive/MyDrive/cursus/uc/model.pkl', 'wb') as f:
  pickle.dump(model, f)

# Make predictions
predictions = []
model.eval()
for batch in test_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy().tolist())

# Save predictions to a CSV file
with open('/content/drive/MyDrive/cursus/uc/intents.csv', 'w') as f:
  f.write('intent\n')
  for prediction in predictions:
    f.write(f'{idx2labels[prediction]}\n')


In [ ]:
pd.read_csv('/content/drive/MyDrive/cursus/uc/intents.csv')


## CNN

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder


df = bikeshop_dataset['train'].to_pandas()
df

In [ ]:
max_len = df["text"].str.split().apply(len).max()
max_len

In [ ]:
# Split the data into input (text) and labels
X = df["text"]
y = df["intent"]

# Create a label encoder to convert labels to numerical values
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Create a custom dataset class
class TextDataset(Dataset):
    def __init__(self, X, y, max_len):
        self.X = X
        self.y = y
        self.max_len = max_len

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        text = self.X.iloc[idx]
        label = self.y[idx]

        # Tokenize the text
        tokens = text.split()

        # Pad the tokens to the maximum length
        tokens = tokens[:self.max_len]
        tokens += ["<PAD>"] * (self.max_len - len(tokens))

        # Convert tokens to numerical values
        tokens = [dataset_vocab.get(token, dataset_vocab["<UNK>"]) for token in tokens]

        # Convert to tensor
        tokens = torch.tensor(tokens)

        return tokens, torch.tensor(label)

# Create a vocabulary
dataset_vocab = {"<PAD>": 0, "<UNK>": 1}
for text in X:
    tokens = text.split()
    for token in tokens:
        if token not in dataset_vocab:
            dataset_vocab[token] = len(dataset_vocab)

In [ ]:
print(dataset_vocab)

In [ ]:
# Create the dataset and data loader
max_len = 32
dataset = TextDataset(X, y_encoded, max_len)
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

In [ ]:
yb

In [ ]:
from dataclasses import dataclass, field

@dataclass
class CNNConfig:

    vocab_size: int = len(dataset_vocab)
    embed_dim: int = 100
    lstm_hidden_dim: int = 100
    fc_hidden_dim: int = 32
    output_dim: int = 9
    dropout: float = 0.5

    num_filters: int = 100
    num_channels: list = field(default_factory=lambda: [100, 100, 100])
    kernel_sizes: list = field(default_factory=lambda: [3, 4, 5]) # Use default_factory for mutable defaults


config = CNNConfig()
config

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def load_glove_embeddings(glove_path, word_to_index, embed_dim):
    embeddings_index = {}
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    embedding_matrix = np.zeros((len(word_to_index), embed_dim))
    for word, i in word_to_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return torch.tensor(embedding_matrix, dtype=torch.float32)

class CNNModel(nn.Module):
    def __init__(self, config: CNNConfig, glove_embeddings, **kwargs):
        super(CNNModel, self).__init__(**kwargs)
        self.embedding = nn.Embedding(config.vocab_size, config.embed_dim)
        self.constant_embedding = nn.Embedding.from_pretrained(glove_embeddings, freeze=True)
        self.dropout = nn.Dropout(0.5)
        self.decoder = nn.Linear(sum(config.num_channels), 9)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.relu = nn.ReLU()
        self.convs = nn.ModuleList()
        for c, k in zip(config.num_channels, config.kernel_sizes):
            self.convs.append(nn.Conv1d(2 * config.embed_dim, c, k))

    def forward(self, inputs):
        # B,T,H -> B,T,H
        embeddings = torch.cat((self.embedding(inputs), self.constant_embedding(inputs)), dim=2)
        embeddings = embeddings.permute(0, 2, 1)
        # For each one-dimensional convolutional layer, after max pooling,
        # a tensor of shape (B, C, 1) is obtained.
        # Remove the last dimension and concatenate along channels
        encoding = torch.cat([
            torch.squeeze(self.relu(self.pool(conv(embeddings))), dim=-1)
            for conv in self.convs], dim=1)
        outputs = self.decoder(self.dropout(encoding))
        return outputs

In [ ]:
glove_embeddings = load_glove_embeddings(f'{dataset_drive_path}/glove/glove.6B.100d.txt', dataset_vocab, config.embed_dim)

In [ ]:
import torch.utils
import torch.utils.data


def train(
    dl_loader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    model: torch.nn.Module,
    ):

    """ Train the model on the training dataset """

    total_loss, total_acc = 0, 0

    model.train()
    for text, label in dl_loader:
        text = text.to(device)
        label = label.to(device)
        # import pdb; pdb.set_trace();
        optimizer.zero_grad()
        preds = model(text)
        loss = criterion(preds, label)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(preds, 1)
        total_loss += loss.item() * label.size(0)
        total_acc += (preds  == label).float().sum().item()
    return total_loss / len(dl_loader.dataset), total_acc / len(dl_loader.dataset)

def evaluate(
    dl_loader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    model: torch.nn.Module,
    ):

    """ Evaluate the model on the validation or test set """

    total_loss, total_acc = 0, 0

    model.eval()
    with torch.inference_mode():
        for text, label in dl_loader:
            preds = model(text)
            loss = criterion(preds, label)
            _, preds = torch.max(preds, 1)
            total_acc += (preds == label).float().sum().item()
            total_loss += loss.item() * label.size(0)
        return total_loss / len(dl_loader.dataset), total_acc / len(dl_loader.dataset)

In [ ]:
cnn_model = CNNModel(config, glove_embeddings).to('cpu')

def init_weights(module):
    if type(module) in (nn.Linear, nn.Conv1d):
        nn.init.xavier_uniform_(module.weight)

cnn_model.apply(init_weights)

print(cnn_model)

In [ ]:
lr, num_epochs = 0.0007, 30
device = torch.device('cpu')
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=lr)


for epoch in range(num_epochs):
    train_loss, train_acc = train(train_loader, optimizer, criteria, cnn_model)
    valid_loss, valid_acc = evaluate(valid_dataloader, criteria, cnn_model)
    print(f'Epoch {epoch}  train loss: {train_loss:.4f} train accuracy: {train_acc:.4f}')
